# Подготовка базы данных

In [1]:
import sqlite3
import pandas as pd

In [2]:
con = sqlite3.connect('window.db')

In [3]:
cursor = con.cursor()

In [4]:
sql = '''

create table activity (
  adate date,
  user_id integer,
  points integer
);

insert into activity values
('2020-02-02', 11, 1),
('2020-02-03', 11, 1),
('2020-02-04', 11, 1),
('2020-02-05', 11, 1),
('2020-02-06', 11, 1),
('2020-02-07', 11, 1),
('2020-02-08', 11, 1),
('2020-02-09', 11, 1),
('2020-02-10', 11, 1),
('2020-02-11', 11, 1),
('2020-02-15', 11, 2),
('2020-02-16', 11, 2),
('2020-02-17', 11, 2),

('2020-02-05', 21, 6),
('2020-02-07', 21, 2),
('2020-02-09', 21, 4),
('2020-02-11', 21, 6),
('2020-02-13', 21, 3),
('2020-02-15', 21, 5),
('2020-02-20', 21, 5),
('2020-02-23', 21, 3),
('2020-02-24', 21, 1),

('2020-02-07', 31, 25),
('2020-02-11', 31, 2),
('2020-02-12', 31, 3),
('2020-02-13', 31, 2),

('2020-02-01', 51, 1),
('2020-02-02', 51, 3),
('2020-02-03', 51, 2),
('2020-02-04', 51, 6),
('2020-02-05', 51, 3),
('2020-02-08', 51, 1),
('2020-02-09', 51, 1),
('2020-02-10', 51, 1),
('2020-02-20', 51, 9),
('2020-02-21', 51, 15),
('2020-02-22', 51, 7),
('2020-02-25', 51, 1);'''

In [5]:
cursor.executescript(sql)

In [7]:
sql = '''select * from activity'''

pd.read_sql(sql, con)

,adate,user_id,points
0,2020-02-02,11,1
1,2020-02-03,11,1
2,2020-02-04,11,1
3,2020-02-05,11,1
4,2020-02-06,11,1
5,2020-02-07,11,1
6,2020-02-08,11,1
7,2020-02-09,11,1
8,2020-02-10,11,1
9,2020-02-11,11,1


In [8]:
# Напишем функцию, чтобы удобнее было возвращать
def select(sql):
    return pd.read_sql(sql, con)

In [9]:
select(sql)

,adate,user_id,points
0,2020-02-02,11,1
1,2020-02-03,11,1
2,2020-02-04,11,1
3,2020-02-05,11,1
4,2020-02-06,11,1
5,2020-02-07,11,1
6,2020-02-08,11,1
7,2020-02-09,11,1
8,2020-02-10,11,1
9,2020-02-11,11,1


# Задача 1

**Посчитайте острова по дате для каждого пользователя — то есть периоды, в которые пользователь набирал хотя бы один балл каждый день без перерыва. Серия из одного дня тоже считается.**

In [12]:
sql = '''

with data as (

select t.*,
unixepoch(t.adate)/86400 as days,
dense_rank() over w as dr,
unixepoch(t.adate)/86400 - dense_rank() over w as diff_group
from activity as t
window w as (
partition by t.user_id
order by t.adate
)

)

select t.user_id,
min(t.adate) as day_start,
max(t.adate) as day_end,
count(*) as day_count
from data as t
group by t.user_id, t.diff_group
order by t.user_id, min(t.adate)
'''
select(sql)

,user_id,day_start,day_end,day_count
0,11,2020-02-02,2020-02-11,10
1,11,2020-02-15,2020-02-17,3
2,21,2020-02-05,2020-02-05,1
3,21,2020-02-07,2020-02-07,1
4,21,2020-02-09,2020-02-09,1
5,21,2020-02-11,2020-02-11,1
6,21,2020-02-13,2020-02-13,1
7,21,2020-02-15,2020-02-15,1
8,21,2020-02-20,2020-02-20,1
9,21,2020-02-23,2020-02-24,2


# Задача 2

**Есть таблица activity, которая хранит баллы, набранные пользователями в конкретные дни. Серией в этой задаче будем считать последовательность дней, в которых количество набранных пользователем очков в каждый следующий день не меньше, чем в предыдущий. При этом между предыдущим и следующим днем может быть любой промежуток времени, это не прерывает серию. Серии из одного дня не учитываем.**

In [13]:
sql = '''
with data as (

select t.adate, t.user_id, t.points,
coalesce(lag(t.points) over w, t.points) as point_lag,
case when (coalesce(lag(t.points) over w, t.points) - t.points) <= 0 then 0 else 1 end as diff
from activity as t
window w as (
partition by t.user_id
order by t.adate)


),

data_2 as (
select t.*,
sum(t.diff) over w as grou from data as t
window w as(
partition by t.user_id
order by t.user_id, t.adate
)

)

select t.user_id,
min(t.adate) as day_start,
max(t.adate) as day_end,
count(*) as day_count,
sum(t.points) as p_total 
from data_2 as t
group by t.user_id, t.grou
having count(*) > 1
order by t.user_id, min(t.adate)

'''
select(sql)

,user_id,day_start,day_end,day_count,p_total
0,11,2020-02-02,2020-02-17,13,16
1,21,2020-02-07,2020-02-11,3,12
2,21,2020-02-13,2020-02-20,3,13
3,31,2020-02-11,2020-02-12,2,5
4,51,2020-02-01,2020-02-02,2,4
5,51,2020-02-03,2020-02-04,2,8
6,51,2020-02-08,2020-02-21,5,27


# Задача 3

In [14]:
sql = '''
drop table if exists numer;

create table numer(
num integer
);
insert into numer values(51);
insert into numer values(52);
insert into numer values(53);
insert into numer values(55);
insert into numer values(56);
insert into numer values(59);
insert into numer values(61);
insert into numer values(72);
'''

In [15]:
con.executescript(sql)

In [16]:
sql = '''

select t.* from numer as t

'''

select(sql)

,num
0,51
1,52
2,53
3,55
4,56
5,59
6,61
7,72


**Вот универсальный алгоритм поиска кластеров на SQL:**

* 1. Посчитать расстояние L между соседними значениями через lag().
* 2. Идентифицировать границы по условию sum(case when L > N then 1 else 0 end), где N — максимально допустимое расстояние между соседними значениями кластера.
* 3. Агрегировать кластеры по идентификатору.

In [17]:
sql = '''
with data as (

select t.*,
(case when t.num - lag(t.num) over w > 2 then 1 else 0 end) as is_boundary,
t.num - lag(t.num) over w as n_lag
from numer as t
window w as (
order by t.num
)
),

data_2 as (

select t.*,
sum(t.is_boundary) over w as group_id
from data as t
window w as(
order by t.num
)

)

select 
min(t.num) as min_num,
max(t.num) as max_num,
count(*)
from data_2 as t
group by t.group_id

'''

select(sql)

,min_num,max_num,count(*)
0,51,56,5
1,59,61,2
2,72,72,1
